# 抓取江苏省政协十年提案

打开 http://www.jszx.gov.cn/zxta/2022ta/

- 点击下一页，url不变!

> 所以数据的更新是使用js推送的
- 分析network中的内容，发现proposalList.jsp
    - 查看它的header，并发现了form_data
    
<img src = './img/form_data.png'>

http://www.jszx.gov.cn/wcm/zxweb/proposalList.jsp 无法在新的tab中打开


根据form data重构url

http://www.jszx.gov.cn/wcm/zxweb/proposalList.jsp?year=2022&pagenum=1&pagesize=20

In [1]:
import requests
from bs4 import BeautifulSoup

In [2]:
form_data = {'year':2022, # change it to the current year
        'pagenum':1,
        'pagesize':20
}
url = 'http://www.jszx.gov.cn/wcm/zxweb/proposalList.jsp'
content = requests.get(url, form_data)
content.encoding = 'utf-8'
js = content.json()

In [3]:
js['data']['totalcount']

'630'

In [5]:
dat = js['data']['list']
pagenum = js['data']['pagecount']
pagenum

32.0

In [6]:
for i in range(2, int(pagenum)+1):
    print(i, end = '\r')
    form_data['pagenum'] = i
    content = requests.get(url, form_data)
    content.encoding = 'utf-8'
    js = content.json()
    for j in js['data']['list']:
        dat.append(j)

In [7]:
len(dat)

630

In [8]:
dat[0]

{'rownum': 1,
 'proposal_number': '0001',
 'reason': '关于深入落实长江大保护战略，推动我省沿江化工产业绿色高质量发展的建议',
 'pkid': 'dd619f014d23456cb403ceb12506739a',
 'year': '2022',
 'publish_time': '2022-01-18 16:12:23',
 'personnel_name': '严华',
 'type': '工业商贸'}

In [9]:
import pandas as pd

df = pd.DataFrame(dat)
df.head()

,rownum,proposal_number,reason,pkid,year,publish_time,personnel_name,type
0,1,0001,关于深入落实长江大保护战略，推动我省沿江化工产业绿色高质量发展的建议,dd619f014d23456cb403ceb12506739a,2022,2022-01-18 16:12:23,严华,工业商贸
1,2,0002,关于重视人工智能应用安全的建议,df4b6c2109af42b2a04b135212923f98,2022,2022-01-18 10:29:37,仲盛,科学技术
2,3,0004,关于打造软件信息产业联动先行区的建议,7f97456a314444c3b59ced0374bb01fc,2022,2022-01-18 16:12:23,钱再见,工业商贸
3,4,0005,关于设立“江苏工匠日”的建议,f5f0aa468ecf4af5be2438393d54a49d,2022,2022-01-18 16:06:13,马永青等9人,文化宣传
4,5,0006,关于进一步重视和支持企业提升人才吸引力的建议,a666191fb1644a5f83009ac1a0dd5e5b,2022,2022-01-19 19:23:47,甘霖,社会事业


In [10]:
df.groupby('type').size()

type
农林水利    69
医卫体育    69
城乡建设    31
工业商贸    89
政治建设    12
教育事业    68
文化宣传    33
法制建设    23
社会事业    92
科学技术    18
经济发展    69
统战综合     5
财税金融    14
资源环境    38
dtype: int64

## 抓取提案内容
http://www.jszx.gov.cn/zxta/2019ta/index_61.html?pkid=18b1b347f9e34badb8934c2acec80e9e

In [11]:
url_base = 'http://www.jszx.gov.cn/wcm/zxweb/proposalInfo.jsp?pkid='
urls = [url_base + i  for i in df['pkid']]

In [18]:
for i in urls[:5]:
    print(i)

http://www.jszx.gov.cn/wcm/zxweb/proposalInfo.jsp?pkid=dd619f014d23456cb403ceb12506739a
http://www.jszx.gov.cn/wcm/zxweb/proposalInfo.jsp?pkid=df4b6c2109af42b2a04b135212923f98
http://www.jszx.gov.cn/wcm/zxweb/proposalInfo.jsp?pkid=7f97456a314444c3b59ced0374bb01fc
http://www.jszx.gov.cn/wcm/zxweb/proposalInfo.jsp?pkid=f5f0aa468ecf4af5be2438393d54a49d
http://www.jszx.gov.cn/wcm/zxweb/proposalInfo.jsp?pkid=a666191fb1644a5f83009ac1a0dd5e5b


In [12]:
text = []
for k, i in enumerate(urls):
    print(k, end = '\r')
    content = requests.get(i)
    content.encoding = 'utf-8'
    js = content.json()
    js = js['data']['binfo']['_content']
    soup = BeautifulSoup(js, 'html.parser') 
    text.append(soup.text)

In [13]:
len(text)

630

In [14]:
df['content'] = text

In [15]:
df.head()

,rownum,proposal_number,reason,pkid,year,publish_time,personnel_name,type,content
0,1,0001,关于深入落实长江大保护战略，推动我省沿江化工产业绿色高质量发展的建议,dd619f014d23456cb403ceb12506739a,2022,2022-01-18 16:12:23,严华,工业商贸,调研情况：化工产业是江苏省支柱产业之一，是我省重要的基础性产业，产业规模、行业基础、技术水平...
1,2,0002,关于重视人工智能应用安全的建议,df4b6c2109af42b2a04b135212923f98,2022,2022-01-18 10:29:37,仲盛,科学技术,调研情况：习近平总书记强调：“人工智能是新一轮科技革命和产业变革的重要驱动力量，加快发展新一...
2,3,0004,关于打造软件信息产业联动先行区的建议,7f97456a314444c3b59ced0374bb01fc,2022,2022-01-18 16:12:23,钱再见,工业商贸,调研情况： 2021年2月8日，南京都市圈发展规划获国家发改委批复，要求以区域间的就近性、互...
3,4,0005,关于设立“江苏工匠日”的建议,f5f0aa468ecf4af5be2438393d54a49d,2022,2022-01-18 16:06:13,马永青等9人,文化宣传,调研情况：近年来，省政协总工会界别委员认真学习贯彻党的十九大精神，围绕省委省政府和省“两会”...
4,5,0006,关于进一步重视和支持企业提升人才吸引力的建议,a666191fb1644a5f83009ac1a0dd5e5b,2022,2022-01-19 19:23:47,甘霖,社会事业,调研情况：为进一步加大对民营经济高质量发展支持力度，我省出台《关于促进民营经济高质量发展的意...


In [17]:
#df.to_csv('./data/jszx2022.csv', index = False)